In [1]:
import sys
import os

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.conformer_small_encoder_config
small_model = conformer.Model(**config)

In [6]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.rnn_multilanguage.Model(small_model, language_size = 2,
                                        vocabulary_size = 100, 
                                        training_mode = True,
                                        **config)

In [7]:
k = 0.0062169171196134855

i = tf.placeholder(tf.float32, [None, None])
lengths = tf.placeholder(tf.float32, [None])
v = tf.expand_dims(tf.map_fn(featurizer.vectorize, i), -1)
length_v = tf.cast(lengths * k, tf.int32)

In [8]:
small_model.conv_subsampling.time_reduction_factor

4

In [9]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [10]:
p = tf.placeholder(tf.int32, [None, None])
p_len = tf.placeholder(tf.int32, [None])
lang = tf.placeholder(tf.int32, [None])

In [11]:
lang_ = tf.tile(tf.expand_dims(lang, -1), (1, tf.shape(p)[1]))
lang_

<tf.Tensor 'Tile:0' shape=(?, ?) dtype=int32>

In [12]:
small_logits, lang_logits = small_transducer([v, p, p_len], training = True)
small_logits, lang_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(<tf.Tensor 'transducer/transducer_joint/mul:0' shape=(?, ?, ?, 100) dtype=float32>,
 <tf.Tensor 'transducer/transducer_joint/transducer_joint_language/Tensordot:0' shape=(?, ?, ?, 2) dtype=float32>)

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [15]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)
t = [[1,1,1,1,1,1], [1,1,1,1,1,1]]

In [16]:
%%time

sess.run(small_logits, feed_dict = {i: padded, lengths: lens, p: t, p_len: [6, 6],
                                   lang: [0, 0]}).shape

CPU times: user 2.04 s, sys: 171 ms, total: 2.21 s
Wall time: 1.73 s


(2, 141, 6, 100)

In [17]:
%%time

sess.run(lang_logits, feed_dict = {i: padded, lengths: lens, p: t, p_len: [6, 6],
                                   lang: [0, 0]}).shape

CPU times: user 2.02 s, sys: 153 ms, total: 2.17 s
Wall time: 1.69 s


(2, 141, 6, 2)

In [18]:
small_transducer.greedy_decoder(v, 
                                length_v // small_model.conv_subsampling.time_reduction_factor,
                                training = False)

(<tf.Tensor 'while/Exit_1:0' shape=(?, ?) dtype=int32>,
 <tf.Tensor 'while/Exit_2:0' shape=(?, ?) dtype=int32>)